*  This program computes 
*   1) in-sample ^2
*   2) out-of-sample R^2  
*  in the predictive regression based on 14 economic predictors 
*  from Goyal's web: http://www.hec.unil.ch/agoyal/ 

In [1]:
import pandas as pd                     # To load data, we use the package pandas
import numpy as np
import statsmodels.api as sm                  # We use this package to do statical estimation
import statsmodels.formula.api as smf


# Load the data and define variables, 1926:12-2010:12
 
df = pd.read_excel('Returns_handbook_Python_data.xlsx', 'Monthly')    # Monthly is the name of Sheet1


   # Market risk premium
market_return = np.array(df.loc[672 : 1679, "CRSP S&P 500 value-weighted return with dividends"])
                                        # 671th row is 1927:01 in the data 
                                       # Excel row is 673 bc/ Python does not count header and starts from 0
r_f_lag = np.array(df.loc[671 : 1678, "Risk-free rate"])
equity_premium = market_return - r_f_lag     # excess return

   # Predictors 

D12 = np.array(df.loc[671 : 1678,   "12-month moving sum of S&P 500 dividends"]); # dividends
SP500 = np.array(df.loc[671 : 1678, "S&P 500 index"]);
DP = np.log(D12) - np.log(SP500);                                # log dividend-price ratio
SP500_lag = np.array(df.loc[671 : 1678, "S&P 500 index"]);   # S&P 500 index, lagged (1926:12-2010:11)
DY = np.log(D12) - np.log(SP500_lag);       # log dividend yield
E12 = np.array(df.loc[671 : 1678,      "12-month moving sum of S&P 500 earnings"]); # earnings
EP = np.log(E12) - np.log(SP500);       # log earnings-price ratio
DE = np.log(D12) - np.log(E12);        # log dividend-payout ratio
SVAR = np.array(df.loc[671 : 1678, "Monthly sum of squared daily returns on S&P 500 index"]); # volatility
BM = np.array(df.loc[671 : 1678, "DJIA book-to-market value ratio"]);   # book-to-market ratio
NTIS = np.array(df.loc[671 : 1678, "Net equity expansion"]); # net equity issuing activity
TBL = np.array(df.loc[671 : 1678, "3-month Treasury bill yield (secondary market)"]); # T-bill rate
LTY = np.array(df.loc[671 : 1678, "Long-term government bond yield"]); # long-term government bond yield
LTR = np.array(df.loc[671 : 1678, "Long-term government bond return"]); # long-term government bond return
TMS = LTY - TBL; # term spread
AAA = np.array(df.loc[671 : 1678, "Moodys AAA-rated corporate bond yield"]); # AAA-rated corporate bond yield
BAA = np.array(df.loc[671 : 1678, "Moodys BAA-rated corporate bond yield"]); # BAA-rated corporate bond yield
DFY = BAA - AAA; # default yield spread
CORPR = np.array(df.loc[671 : 1678, "Long-term corporate bond return"]); # long-term corporate bond return
DFR = CORPR - LTR; # default return spread
INFL_lag = np.array(df.loc[671 : 1678, "CPI (all urban consumers) inflation rate"]); # inflation, lagged (1926:12-2010:11)

In [2]:
# Compute Computing regression slope and R^2 

 
N = 14;                             
T = 1008  
ECON = np.vstack((DP, DY, EP, DE, SVAR, BM, NTIS, TBL, LTY, LTR, TMS, DFY, DFR, INFL_lag)).T; 
                            # print(ECON.shape), one should get (1008* 14)

y = np.array(equity_premium)
y.shape = (T,1)                 # make sure the dimentionality

onesT= np.ones((T,1))          # The constant part

coeff = np.ones((N,2))           # to store all the alphas and betas
tValues = np.ones((N,2))  
R2a = np.ones((N,1))

for i in range(N):
    x = ECON[:,i]                       # i-th predictor 
    x = np.array(x)
    x.shape = (T,1)
    xx = np.hstack((onesT, x))          # add the constant part to x
    reg = sm.OLS(endog=y, exog=xx)
    results = reg.fit()
    coeff[i,:] = results.params           # paramter estimates, output of sm.OLS
    tValues[i,:] = results.tvalues
    R2a[i] = results.rsquared_adj

slope = coeff[:,1].reshape(-1, 1)             # another to make it N by 1 vector
slopeTvalue = tValues[:,1].reshape(-1, 1)
 
Output = np.hstack((slope,slopeTvalue, R2a))
print(' Slope, t-value, Adjusted R-sqaured   \n')
print(Output)

 Slope, t-value, Adjusted R-sqaured   

[[ 8.71690626e-03  2.23407276e+00  3.94769168e-03]
 [ 8.71690626e-03  2.23407276e+00  3.94769168e-03]
 [ 8.79402795e-03  2.12372722e+00  3.47370786e-03]
 [ 1.75690485e-03  3.22423810e-01 -8.90606662e-04]
 [-2.16752038e-01 -6.16069360e-01 -6.16525394e-04]
 [ 1.95321487e-02  2.95791580e+00  7.63663115e-03]
 [-1.44982499e-01 -2.02180170e+00  3.05684563e-03]
 [-9.68401754e-02 -1.68850653e+00  1.83481425e-03]
 [-7.51328677e-02 -1.19804056e+00  4.32088482e-04]
 [ 1.11515491e-01  1.51476612e+00  1.28386735e-03]
 [ 1.85987941e-01  1.38414062e+00  9.08652495e-04]
 [ 4.09313656e-01  1.66105113e+00  1.74381662e-03]
 [ 1.37088077e-01  1.02480437e+00  4.98723851e-05]
 [-4.94960157e-01 -1.48893255e+00  1.20700230e-03]]


In [3]:
#==============================================================================
# Define a function to calculate OOS R2
#==============================================================================
def R2OOS(r_real, r_hat, r_bar):
    denominator_res=(r_real-r_bar)**2
    denominator=np.sum(denominator_res)
    numerator_res=(r_real-r_hat)**2
    numerator=np.sum(numerator_res)
    r2oos=1-numerator/denominator   
    return r2oos

In [4]:
r2oos = []
training_period = 120 # use the first 10 year's data as training data set

for i in range(N):
    x = ECON[:,i]  # i-th indicator
    y_hat = []     # create a variable to collect prediction
    y_bar = []     # create a variable to collect rolling average
    for j in range(training_period,len(x)):    
        rhs = x[:j-1]              # from 0 to j-1 as explantory variable
        rhs = sm.add_constant(rhs) # add constant 1 to rhs variable
        lhs = y[1:j]               # from 1 to j as explained variable
        reg = sm.OLS(endog=lhs, exog=rhs)
        result = reg.fit()
        y_hat.append(result.fittedvalues[-1]) # use the last fitted values as the rolling out-of-sample prediction
        y_bar.append(np.mean(lhs))            # use the rolling average as the out-of-sample average prediction
    
    y_hat = np.array(y_hat)
    y_bar = np.array(y_bar)
    y_real = y[-len(y_hat):]
    r2oos.append(R2OOS(y_real, y_hat, y_bar))

r2oos = np.array(r2oos)
r2oos.shape = (N,1)

print(' Out-of-sample R^2 in % \n')
print(r2oos)

 Out-of-sample R^2 in % 

[[-0.02110615]
 [-0.02110615]
 [-0.0735914 ]
 [-0.00552625]
 [-0.00040377]
 [-0.04149747]
 [-0.01600652]
 [-0.01224885]
 [-0.02249995]
 [-0.00545114]
 [-0.00603053]
 [-0.00671591]
 [-0.00966326]
 [-0.0021698 ]]
